In [1]:
%load_ext autoreload
%autoreload 2
import sys,os
sys.path.append('../')   
import numpy as np
import pandas as pd
from model.erroneousPreference import erroneousPreference
from kernel import RBF, Linear, LinearRBF
from utility import  paramz
from sklearn.model_selection import train_test_split, GroupShuffleSplit
# for plotting
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import arviz as az
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GroupShuffleSplit 

In [2]:
df = pd.read_csv("datasets/train.csv",index_col=0)
df.reset_index(drop=True,inplace=True)
df.drop(columns=['choiceid'],inplace=True)
print(df.shape)
df.head(10)
print(df[(df.id==1)|(df.id==120)].to_latex(index=False))

(2929, 10)
\begin{tabular}{rlrrrrrrrr}
\toprule
 id & choice &  price\_A &  time\_A &  change\_A &  comfort\_A &  price\_B &  time\_B &  change\_B &  comfort\_B \\
\midrule
  1 &      A &     2400 &     150 &         0 &          1 &     4000 &     150 &         0 &          1 \\
  1 &      A &     2400 &     150 &         0 &          1 &     3200 &     130 &         0 &          1 \\
  1 &      A &     2400 &     115 &         0 &          1 &     4000 &     115 &         0 &          0 \\
  1 &      B &     4000 &     130 &         0 &          1 &     3200 &     150 &         0 &          0 \\
  1 &      B &     2400 &     150 &         0 &          1 &     3200 &     150 &         0 &          0 \\
  1 &      B &     4000 &     115 &         0 &          0 &     2400 &     130 &         0 &          0 \\
  1 &      B &     2400 &     150 &         0 &          1 &     3200 &     115 &         0 &          1 \\
  1 &      B &     2400 &     115 &         0 &          1 &     3200 &

<ipython-input-2-d559d8f9b299>:6: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df[(df.id==1)|(df.id==120)].to_latex(index=False))


In [3]:

# transform covariates
mean_price = df[['price_A','price_B']].mean()
std_price = np.sqrt(df[['price_A','price_B']].var())
mean_time = df[['time_A','time_B']].mean()
std_time = np.sqrt(df[['time_A','time_B']].var())
df.loc[:,['price_A','price_B']] =(df.loc[:,['price_A','price_B']]-mean_price)/std_price
df.loc[:,['time_A','time_B']] =(df.loc[:,['time_A','time_B']]-mean_time)/std_time
df.head()

,id,choice,price_A,time_A,change_A,comfort_A,price_B,time_B,change_B,comfort_B
0,1,A,-0.746433,0.772057,0,1,0.496535,0.816768,0,1
1,1,A,-0.746433,0.772057,0,1,-0.131240,0.102026,0,1
2,1,A,-0.746433,-0.428456,0,1,0.496535,-0.434031,0,0
3,1,B,0.487528,0.086050,0,1,-0.131240,0.816768,0,0
4,1,B,-0.746433,0.772057,0,1,-0.131240,0.816768,0,0


## Split train and test

In [4]:
splitter = GroupShuffleSplit(test_size=0.3, n_splits=1, random_state = 10)
split = splitter.split(df, groups=df['id'])
ind_train, ind_test = next(split)

df_train = df.iloc[ind_train,:]
df_test  = df.iloc[ind_test,:]
df_train.shape

(2044, 10)

## Make preferences

In [5]:
def compute_pair(df):
    Xa = np.unique(df[['price_A', 'time_A', 'change_A',
       'comfort_A']],axis=0)
    Xb = np.unique(df[['price_B', 'time_B', 'change_B',
           'comfort_B']],axis=0)
    X = np.unique(np.vstack([Xa,Xb]),axis=0)# objects
    Pair = []
    for el in range(0,df.shape[0]):
        rowi=df.iloc[el][['price_A', 'time_A', 'change_A',
           'comfort_A']]
        rowj=df.iloc[el][['price_B', 'time_B', 'change_B',
           'comfort_B']]
        i = np.where( np.isin(X, rowi).all(axis=1))[0][0]
        j = np.where( np.isin(X, rowj).all(axis=1))[0][0]
        if df.iloc[el].choice=="A":
            Pair.append([i,j])
        else:
            Pair.append([j,i])
    Pair=np.vstack(Pair)
    return Pair, X


Pair_tr,X_tr = compute_pair(df_train)
Pair_te,X_te = compute_pair(df_test)


In [64]:
lengthscale = np.array([2.46811807e+00, 1.06970835e+01, 9.61723050e-03, 4.12986443e+00])
variance = 14.0
lengthscale = np.array([2.45749795e+00, 6.42781558e+00, 1.05724232e+01, 3.07650136e-03])
variance = 7.28
#lengthscale =np.array([4.27103877e+00, 1.06897984e+01, 9.97192222e-06, 4.78859211e+00])
#variance = 10.0
#0.1+np.random.rand(1,data["X"].shape[1]) 
#lengthscale = np.array([3.14554881e+01, 6.84333683e-04, 1.58214102e-04,   2.54469097e+01])

In [65]:
# data dictionary
data = {}
data["Pairs"] = Pair_tr
data["X"] = X_tr

# define kernel and hyperparams
Kernel = RBF

# kernel parameter dictionary
params={}
params['lengthscale']={'value':lengthscale, #0.1+np.random.rand(1,data["X"].shape[1]) ,
                            'range':np.vstack([[1e-6, 40.0]]*data["X"].shape[1]),
                            'transform': paramz.logexp()}
params['variance']={'value':np.array([variance]), 
                            'range':np.vstack([[0.1, 80.0]]),
                            'transform': paramz.logexp()}



# define preference model 
model = erroneousPreference(data,Kernel,params,inf_method="laplace",jitter=1e-6)
# compute hyperparameters
model.optimize_hyperparams(num_restarts=1,niterations=50) 
print(model.params)
# sample from posterior
#model.sample(nsamples=5000, tune=500)

  0%|          | 0/50 [00:00<?, ?it/s]

Iteration 0  -871.6485096593259
{'lengthscale': {'value': array([3.46730582e+00, 7.03373565e+00, 1.07750639e+01, 3.07650136e-03]), 'range': array([[1.e-06, 4.e+01],
       [1.e-06, 4.e+01],
       [1.e-06, 4.e+01],
       [1.e-06, 4.e+01]]), 'transform': <utility.paramz.logexp object at 0x7fd0d42cef10>}, 'variance': {'value': array([8.83376785]), 'range': array([[ 0.1, 80. ]]), 'transform': <utility.paramz.logexp object at 0x7fd0d4eddc70>}}


In [ ]:
varianceLin = np.array([[0.88162216, 0.15630346, 0.02796235, 0.22389063]])

In [27]:
# data dictionary
data = {}
data["Pairs"] = Pair_tr
data["X"] = X_tr

# define kernel and hyperparams
Kernel = Linear

# kernel parameter dictionary
params={}
params['variance']={'value':0.1+np.random.rand(1,data["X"].shape[1]) ,
                            'range':np.vstack([[1e-6, 40.0]]*data["X"].shape[1]),
                            'transform': paramz.logexp()}




# define preference model 
model = erroneousPreference(data,Kernel,params,inf_method="laplace",jitter=1e-6)
# compute hyperparameters
model.optimize_hyperparams(num_restarts=3,niterations=50) 
print(model.params)
# sample from posterior
#model.sample(nsamples=5000, tune=500)

  0%|          | 0/50 [00:00<?, ?it/s]

Iteration 0  -893.200436845217


  0%|          | 0/50 [00:00<?, ?it/s]

Iteration 1  -893.2004304125262


  0%|          | 0/50 [00:00<?, ?it/s]

Iteration 2  -900.9551234605908
{'variance': {'value': array([[0.88162216, 0.15630346, 0.02796235, 0.22389063]]), 'range': array([[1.e-06, 4.e+01],
       [1.e-06, 4.e+01],
       [1.e-06, 4.e+01],
       [1.e-06, 4.e+01]]), 'transform': <utility.paramz.logexp object at 0x7fd0e0f49070>}}


In [6]:
# data dictionary
data = {}
data["Pairs"] = Pair_tr
data["X"] = X_tr

# define kernel and hyperparams
Kernel = LinearRBF

# kernel parameter dictionary
params={}
params['lin_variance']={'value':0.1+np.random.rand(1,data["X"].shape[1]) ,
                            'range':np.vstack([[1e-6, 40.0]]*data["X"].shape[1]),
                            'transform': paramz.logexp()}

params['rbf_lengthscale']={'value':0.1+np.random.rand(1,data["X"].shape[1]) , #0.1+np.random.rand(1,data["X"].shape[1]) ,
                            'range':np.vstack([[1e-6, 40.0]]*data["X"].shape[1]),
                            'transform': paramz.logexp()}
params['rbf_variance']={'value':np.array([1.0]), 
                            'range':np.vstack([[0.1, 80.0]]),
                            'transform': paramz.logexp()}


# define preference model 
model = erroneousPreference(data,Kernel,params,inf_method="laplace",jitter=1e-6)
# compute hyperparameters
model.optimize_hyperparams(num_restarts=15,niterations=50) 
print(model.params)
# sample from posterior
#model.sample(nsamples=5000, tune=500)

  0%|          | 0/50 [00:00<?, ?it/s]

/home/benavoli/Dropbox/DropBoxRunning/SkewNormal_all/Python_preferences_GP/GPpref/notebooks/../inference/laplace_generic.py:48: RuntimeWarning: invalid value encountered in matmul
  K_Wi_i = LiW12.T@LiW12


Iteration 0  -1211.4642031294281


  0%|          | 0/50 [00:00<?, ?it/s]

Iteration 1  -1224.291978162989


...

Iteration 12  -1250.175459038039


  0%|          | 0/50 [00:00<?, ?it/s]

Iteration 13  -1227.1345330364786


  0%|          | 0/50 [00:00<?, ?it/s]

Iteration 14  -1246.0753520949643
{'lin_variance': {'value': array([[4.53165414, 0.31031663, 0.10059361, 1.12900372]]), 'range': array([[1.e-06, 4.e+01],
       [1.e-06, 4.e+01],
       [1.e-06, 4.e+01],
       [1.e-06, 4.e+01]]), 'transform': <utility.paramz.logexp object at 0x7f0ae92e6f70>}, 'rbf_lengthscale': {'value': array([[0.05234412, 0.37084013, 0.65467292, 0.11223501]]), 'range': array([[1.e-06, 4.e+01],
       [1.e-06, 4.e+01],
       [1.e-06, 4.e+01],
       [1.e-06, 4.e+01]]), 'transform': <utility.paramz.logexp object at 0x7f0ae92e6dc0>}, 'rbf_variance': {'value': array([0.36557808]), 'range': array([[ 0.1, 80. ]]), 'transform': <utility.paramz.logexp object at 0x7f0ae92e6df0>}}


In [ ]:
{'lin_variance': {'value': array([[1.28555553, 0.353382  , 0.07053683, 0.78240802]]), 'range': array([[1.e-06, 4.e+01],
       [1.e-06, 4.e+01],
       [1.e-06, 4.e+01],
       [1.e-06, 4.e+01]]), 'transform': <utility.paramz.logexp object at 0x7fd0d4e2c340>}, 'rbf_lengthscale': {'value': array([[1.2273808 , 0.86528886, 1.76259607, 0.97906652]]), 'range': array([[1.e-06, 4.e+01],
       [1.e-06, 4.e+01],
       [1.e-06, 4.e+01],
       [1.e-06, 4.e+01]]), 'transform': <utility.paramz.logexp object at 0x7fd0d4e2c910>}, 'rbf_variance': {'value': array([0.47640206]), 'range': array([[ 0.1, 80. ]]), 'transform': <utility.paramz.logexp object at 0x7fd0d4e2cee0>}

In [66]:
model.sample(nsamples=2000, tune=200)

100%|██████████| 2200/2200 [03:26<00:00, 10.67it/s]


In [67]:
upred = np.mean(model.predict(X_te),axis=1)
upred.shape

(2031,)

In [68]:
Acc = []
for i in range(Pair_te.shape[0]):
    if upred[Pair_te[i][0]]>upred[Pair_te[i][1]]:
        Acc.append(1.0)
    else:
        Acc.append(0.0)
np.mean(Acc)

0.7119453924914676

In [6]:
pip install numpy==1.24

  Using cached numpy-1.24.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
DEPRECATION: graphql-ws 0.3.0 has a non-standard dependency specifier graphql-core>=2.0<3. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of graphql-ws or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.2
    Uninstalling numpy-1.26.2:
      Successfully uninstalled numpy-1.26.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ctgan 0.5.1 requires packaging<22,>=20, but you have packaging 23.2 which is incompatible.
gluonts 0.8.1 requires typing-extensions~=3.10.0.0, but you have typing-extensions 4.3.0 which is incompatible.


In [7]:
import pandas as pd
import numpy as np 
import GPy as GPy
from GPy_preferenceKernels import PreferenceKern
from sklearn.model_selection import train_test_split

In [14]:
df_train

,id,choice,price_A,time_A,change_A,comfort_A,price_B,time_B,change_B,comfort_B
0,1,A,-0.746433,0.772057,0,1,0.496535,0.816768,0,1
1,1,A,-0.746433,0.772057,0,1,-0.131240,0.102026,0,1
2,1,A,-0.746433,-0.428456,0,1,0.496535,-0.434031,0,0
3,1,B,0.487528,0.086050,0,1,-0.131240,0.816768,0,0
4,1,B,-0.746433,0.772057,0,1,-0.131240,0.816768,0,0
...,...,...,...,...,...,...,...,...,...,...
2884,232,A,0.179038,-0.599958,0,1,0.182648,-0.076660,1,0
2885,232,B,0.487528,-0.085452,1,1,-0.131240,0.459397,1,1
2886,232,B,0.179038,0.429053,1,1,0.496535,-0.076660,1,1
2887,232,A,0.487528,-0.599958,1,1,-0.131240,0.459397,1,1


In [15]:
X_tr = df_train.iloc[:,2:].values
X_te =  df_test.iloc[:,2:].values




def random_swap(X0,seed=1):
    #random_swap
    np.random.seed(seed)
    X=X0.copy()
    Y=np.zeros((X.shape[0],1))
    for i in range(X.shape[0]):
        if np.random.rand(1)<0.5:
            tmp = X[i,0:4].copy()
            X[i,0:4]=X[i,4:].copy()
            X[i,4:] = tmp.copy()
            Y[i]=0
        else:
            Y[i]=1
    return X,Y
        
#random_swap
X_tr_n,Y_tr = random_swap(X_tr,1)
X_te_n,Y_te = random_swap(X_te,2)




In [24]:
lengthscale =np.array([1,1,1,1])
variance=1.0
kernel = PreferenceKern(X_tr_n.shape[1], name="GeneralPreferenceKern", 
                        lengthscale=lengthscale, variance=variance, ARD=True)
#kernel = GPy.kern.Linear(X_tr_n.shape[1],ARD=True)
inference  = GPy.inference.latent_function_inference.Laplace()
likelihood = GPy.likelihoods.Bernoulli()
# Model definition

m = GPy.core.GP(
    X_tr_n,Y_tr, kernel=kernel, likelihood=likelihood, inference_method=inference)
m.kern.lengthscale.constrain_bounded(1e-5,40)
for i in range(10):
    try:
        
        m.optimize_restarts(1)
        print(repr(m.param_array))
        m.randomize()
    except:
        print("some problem")

reconstraining parameters gp.GeneralPreferenceKern.lengthscale


Optimization restart 1/1, f = 1415.4074888930652
array([ 0.04455347, 39.99652876,  1.10672021,  1.83347078, 39.99556748])
some problem
updates were off, setting updates on again


 /home/benavoli/anaconda3/envs/myenv/lib/python3.9/site-packages/GPy/core/__init__.py:80: FutureWarning:In the future `np.bool` will be defined as the corresponding NumPy scalar.  (This may have returned Python scalars in past versions.


KeyboardInterrupt caught, calling on_optimization_end() to round things up
some problem
Optimization restart 1/1, f = 1416.3044639091827
array([ 0.06637614, 39.95799023, 39.97595336,  4.91815811, 39.95152845])
some problem
updates were off, setting updates on again
Optimization restart 1/1, f = 1416.304680552866
array([ 0.06640212, 39.89999479, 39.90464412,  4.93770958, 39.6535146 ])
some problem
updates were off, setting updates on again
Optimization restart 1/1, f = 1416.3045025709898
array([ 0.06670356, 39.91790327, 39.91379799,  4.94381746, 39.95485447])
some problem
updates were off, setting updates on again
KeyboardInterrupt caught, calling on_optimization_end() to round things up
some problem
KeyboardInterrupt caught, calling on_optimization_end() to round things up
some problem
KeyboardInterrupt caught, calling on_optimization_end() to round things up
some problem
KeyboardInterrupt caught, calling on_optimization_end() to round things up
some problem
KeyboardInterrupt caught, c

In [22]:
Y_pred=m.predict(X_te_n)[0]
np.mean((Y_pred>0.5)==Y_te)

0.48361581920903957

False

In [250]:
m.predict(X_te_n)

(array([[0.078184  ],
        [0.91605766],
        [0.05717934],
        [0.08390796],
        [0.07551436],
        [0.09316835],
        [0.060072  ],
        [0.06612089],
        [0.07558736],
        [0.9288635 ],
        [0.31934312],
        [0.7697965 ],
        [0.42635585],
        [0.66943297],
        [0.42210054],
        [0.78558023],
        [0.35076472],
        [0.5       ],
        [0.43548261],
        [0.41594431],
        [0.8241773 ],
        [0.81577496],
        [0.19019815],
        [0.83314615],
        [0.76768611],
        [0.8866981 ],
        [0.21787981],
        [0.12819139],
        [0.15655527],
        [0.6949131 ],
        [0.104309  ],
        [0.17880167],
        [0.88913212],
        [0.92652544],
        [0.88808796],
        [0.18194804],
        [0.74974725],
        [0.78779424],
        [0.05930568],
        [0.90138765],
        [0.92108561],
        [0.92099835],
        [0.05911075],
        [0.88508589],
        [0.10103457],
        [0